## 운동 분류

In [1]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from gcn_source import GraphRNNClassifier
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle

## data 

In [2]:
with open("start_dataloader.pkl", "rb") as f:
    saved_data = pickle.load(f)

loaded_dataset = saved_data["dataset"]
start_dataloader = DataLoader(
    loaded_dataset,
    batch_size=saved_data["batch_size"],
    shuffle=saved_data["shuffle"],
)
for inputs, labels in start_dataloader:
    print(inputs.shape)  # 모델의 입력 크기 확인
    break


torch.Size([4, 30, 1, 640, 480])


In [3]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
from cnn_source import Simple3DCNN
model = Simple3DCNN().to(device)
optimizer = optim.RAdam(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(optimizer, T_max=25, eta_min=0.0001)
def saveModel():
    torch.save(model.state_dict(), f'model/classifying_2.pt')
loss_ = []
valoss_ = []
logger = {"train_loss": list(),
          "validation_loss": list(),

          }

device

device(type='cuda', index=0)

## 모든 데이터 train

In [ ]:
model.train()  # 학습 모드 설정
epochs = 30
min_loss = 1e+10
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(start_dataloader):
        # 데이터를 디바이스로 이동
        inputs = inputs.to(device).permute(0,2,1,3,4)
        labels = labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()
        
        # 모델 출력 계산
        outputs = model(inputs)
        
        # 손실 계산
        loss = criterion(outputs, labels)
        
        # 역전파 및 옵티마이저 업데이트
        loss.backward()
        optimizer.step()
        
        # 배치 손실 기록
        running_loss += loss.item()
        mean_loss = running_loss / len(start_dataloader)
    # 에포크별 평균 손실 출력
    print(f"Epoch {epoch+1}, Loss: {mean_loss}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
print("학습 완료")


Epoch 1, Loss: 0.0002988682945078024
model_saved
Epoch 2, Loss: 0.00010496939752234492
model_saved
Epoch 3, Loss: 7.315874366589917e-05
model_saved
Epoch 4, Loss: 5.301659988059321e-05
model_saved
Epoch 5, Loss: 3.9623061668104186e-05
model_saved
Epoch 6, Loss: 2.6517843248292963e-05
model_saved
Epoch 7, Loss: 8.836745602475798e-06
model_saved
Epoch 8, Loss: 4.319306763976485e-06
model_saved
Epoch 9, Loss: 2.7277643357768815e-06
model_saved
Epoch 10, Loss: 2.1074811572020486e-06
model_saved
Epoch 11, Loss: 1.6715407561144604e-06
model_saved
Epoch 12, Loss: 1.4101840676265466e-06
model_saved
Epoch 13, Loss: 1.2332204908234286e-06
model_saved
Epoch 14, Loss: 1.06997172108811e-06
model_saved
Epoch 15, Loss: 8.951169856696959e-07
model_saved
Epoch 16, Loss: 7.724809912207119e-07
model_saved
Epoch 17, Loss: 6.793832235192778e-07
model_saved
Epoch 18, Loss: 5.912962880468329e-07
model_saved
Epoch 19, Loss: 5.203513487941288e-07
model_saved
Epoch 20, Loss: 4.6997811466742323e-07
model_saved
